### Exploration of results

Analysis of prediction quality of trading sidnals

In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'c:\\Users\\benny\\OneDrive\\Studium\\Tübingen\\DS_in_B&E\\Masterarbeit\\code'

- Comprehensible list to see the specific timeframe in which a stock was part of the analysis

In [3]:
def extract_signal(text):
    # Normalize text: lowercase, remove punctuation, remove extra spaces
    text_clean = ''.join(c for c in text.lower())
    
    # Map strong signals to regular ones
    if 'strong buy' in text_clean:
        return 'buy'
    if 'strong sell' in text_clean:
        return 'sell'
    if 'buy' in text_clean:
        return 'buy'
    if 'sell' in text_clean:
        return 'sell'
    if 'hold' in text_clean:
        return 'hold'
    return None 

In [4]:
# List of companies to show when they were present in the dataset
ciks_and_dates = pd.read_csv("../results/shared_ciks_and_dates.csv", dtype ={"cik": str})
# Convert date column to datetime for analysis
ciks_and_dates["date"] = pd.to_datetime(ciks_and_dates["date"])

In [5]:
# For every CIK, find the first and last date they appear in the dataset
cik_date_ranges = ciks_and_dates.groupby("cik")["date"].agg(["min", "max"]).reset_index()
cik_date_ranges.columns = ["cik", "first_date", "last_date"]
cik_date_ranges["first_date"] = cik_date_ranges["first_date"].dt.strftime('%m-%Y')
cik_date_ranges["last_date"] = cik_date_ranges["last_date"].dt.strftime('%m-%Y')

In [6]:
stock_prices = pd.read_csv("../data/sp1500_monthly_prices.csv", dtype={"cik": str})
stock_prices["date"] = pd.to_datetime(stock_prices["date"])
stock_prices["date"] = stock_prices["date"].dt.strftime('%m-%Y')

In [7]:
cik_date_ranges = cik_date_ranges.merge(stock_prices[["cik", "security"]],on=["cik"], how="left").drop_duplicates(subset = ["cik"]).reset_index(drop=True)
cik_date_ranges.to_csv("../results/analysis_window_by_cik.csv", index=False)

- Simple check, if Analyst/LLM recommendations predicted a rising/falling stock price correctly i.e. buy/hold/sell was recommended

In [8]:
# Load in LLM and Analyst ratings and format date accordingly
llm_ratings = pd.read_csv("../results/llm_recommendations_final.csv", dtype={"cik": str}, low_memory=False)
llm_ratings["date"] = pd.to_datetime(llm_ratings["date"])
llm_ratings["date"] = llm_ratings["date"].dt.strftime('%m-%Y')
llm_ratings["action"] = llm_ratings["rating"].apply(extract_signal)
analyst_ratings = pd.read_csv("../results/sp1500_sell_side_recommendations_ffilled.csv", dtype={"cik": str}, low_memory=False).drop(columns=["mean_rating", "rating", "security"], axis = 1)
analyst_ratings["date"] = pd.to_datetime(analyst_ratings["date"])
analyst_ratings["date"] = analyst_ratings["date"].dt.strftime('%m-%Y')

In [9]:
llm_ratings

,cik,date,rating,action
0,0001497645,03-2010,sell,sell
1,0001497645,06-2010,sell,sell
2,0001497645,09-2010,sell,sell
3,0001497645,12-2010,sell,sell
4,0001497645,03-2011,sell,sell
...,...,...,...,...
147468,0001822993,12-2022,strong sell,sell
147469,0001822993,03-2023,sell,sell
147470,0001822993,06-2023,buy,buy
147471,0001822993,09-2023,buy,buy


In [10]:
# Merge both with stock prices
llm_ratings = llm_ratings.merge(stock_prices, on=["cik", "date"], how="left").drop(columns = ["security"], axis = 1)
analyst_ratings = analyst_ratings.merge(stock_prices, on=["cik", "date"], how="left").drop(columns = ["security"], axis = 1)
# Drop NAs
llm_ratings = llm_ratings.dropna()
analyst_ratings = analyst_ratings.dropna()

In [11]:
# Determine which ciks are present in both datasets on same date
shared_ciks_and_dates = pd.merge(llm_ratings[["cik", "date"]], analyst_ratings[["cik", "date"]], on=["cik", "date"], how="inner").drop_duplicates().reset_index(drop=True)

In [12]:
# Only keep ratings for those ciks and dates
llm_ratings = llm_ratings.merge(shared_ciks_and_dates, on=["cik", "date"], how="inner").reset_index(drop=True).drop_duplicates(subset = ["cik", "date"]).reset_index(drop=True)
analyst_ratings = analyst_ratings.merge(shared_ciks_and_dates, on=["cik", "date"], how="inner").reset_index(drop=True).drop_duplicates(subset = ["cik", "date"]).reset_index(drop=True)

In [13]:
# Shift recommendationms by one month to align with future stock returns
llm_ratings["next_price"] = llm_ratings.groupby("cik")["price"].shift(-1)
analyst_ratings["next_price"] = analyst_ratings.groupby("cik")["price"].shift(-1)

In [ ]:
def compute_actual_movement(row):
    change = (row['next_price'] - row['price']) / row['price']
    if abs(change) <= 0.02: return 'hold'
    return 'buy' if change > 0 else 'sell'

In [ ]:
# Determine if recommendation was "correct"
# Logic: If signal was "buy" and the following price is higher, the recommendation was correct.
# If signal was "sell" and the following price is lower, the recommendation was correct.
# If signal was "hold" and the following price is within +-2% of the current price, the recommendation was correct.
llm_ratings["rec_correct"] = (
    ((llm_ratings["next_price"] > llm_ratings["price"]) & (llm_ratings["action"] == "buy")) | 
    ((llm_ratings["next_price"] < llm_ratings["price"]) & (llm_ratings["action"] == "sell")) | 
    (((llm_ratings["next_price"] - llm_ratings["price"]).abs() / llm_ratings["price"] <= 0.02) & (llm_ratings["action"] == "hold"))
)
# Add actual movement column
llm_ratings["actual_movement"] = llm_ratings.apply(compute_actual_movement, axis=1)
llm_ratings

,cik,date,rating,action,price,next_price,rec_correct,actual_movement
0,0001497645,03-2011,sell,sell,9.94,11.35,False,buy
1,0001497645,06-2011,sell,sell,11.35,7.06,True,sell
2,0001497645,09-2011,sell,sell,7.06,9.44,False,buy
3,0001497645,12-2011,sell,sell,9.44,7.58,True,sell
4,0001497645,03-2012,sell,sell,7.58,8.37,False,buy
...,...,...,...,...,...,...,...,...
111459,0001822993,12-2022,strong sell,sell,34.79,37.41,False,buy
111460,0001822993,03-2023,sell,sell,37.41,30.61,True,sell
111461,0001822993,06-2023,buy,buy,30.61,38.22,True,buy
111462,0001822993,09-2023,buy,buy,38.22,51.20,True,buy


Why next price missing fo 001497645 on 12-2023 since price for 03-2000 is present?

In [16]:
analyst_ratings ["rec_correct"] = (
    ((analyst_ratings["next_price"] > analyst_ratings["price"]) & (analyst_ratings["action"] == "buy")) |
    ((analyst_ratings["next_price"] < analyst_ratings["price"]) & (analyst_ratings["action"] == "sell")) | 
    (((analyst_ratings["next_price"] - analyst_ratings["price"]).abs() / analyst_ratings["price"] <= 0.02) & (analyst_ratings["action"] == "hold"))
)
# Add actual movement column
analyst_ratings["actual_movement"] = analyst_ratings.apply(compute_actual_movement, axis=1)
analyst_ratings

,cik,date,action,price,next_price,rec_correct,actual_movement
0,0000001750,02-2000,buy,23.750,13.875,False,sell
1,0000001750,05-2000,buy,13.875,11.250,False,sell
2,0000001750,08-2000,buy,11.250,10.375,False,sell
3,0000001750,11-2000,buy,10.375,13.600,True,buy
4,0000001750,02-2001,buy,13.600,14.000,True,hold
...,...,...,...,...,...,...,...
111459,0002012383,12-2022,buy,708.630,669.120,False,sell
111460,0002012383,03-2023,buy,669.120,691.140,True,hold
111461,0002012383,06-2023,buy,691.140,646.490,False,sell
111462,0002012383,09-2023,buy,646.490,811.800,True,buy


In [17]:
print(f"Accuracy of LLM recommendations: {llm_ratings['rec_correct'].mean()}")
print(f"Accuracy of Analyst recommendations: {analyst_ratings['rec_correct'].mean()}")

Accuracy of LLM recommendations: 0.47093231895499893
Accuracy of Analyst recommendations: 0.4320856958300438


In [23]:
from sklearn.metrics import classification_report
def get_formatted_report(df):

    # Copy of df with no NAs in actual_movement or action
    eval_df = df.dropna(subset=['actual_movement', 'action']).copy()
    
    # Report as dictionary, so that it can easier be manipulated
    report_dict = classification_report(
        eval_df['actual_movement'], 
        eval_df['action'], 
        output_dict=True
    )
    
    # No averages needed
    keys_to_remove = ['accuracy', 'macro avg', 'weighted avg']
    for key in keys_to_remove:
        report_dict.pop(key, None)
        
    # Back to DataFrame for better formatting
    report_df = pd.DataFrame(report_dict).transpose()
    # Capitalize everything
    report_df.index = [str(i).capitalize() for i in report_df.index]
    report_df.columns = [str(i).capitalize() for i in report_df.columns]
    # Drop number of samples column and rounding
    report_df = report_df.drop(columns=['Support'], axis=1)
    report_df = report_df.round(4)
    
    return report_df

In [24]:
llm_classification_report = get_formatted_report(llm_ratings)
llm_classification_report.to_csv("../results/llm_classification_report.csv")
llm_classification_report

,Precision,Recall,F1-score
Buy,0.4665,0.4418,0.4538
Hold,0.2868,0.1406,0.1886
Sell,0.3340,0.5131,0.4047


In [25]:
analyst_classification_report = get_formatted_report(analyst_ratings)
analyst_classification_report.to_csv("../results/analyst_classification_report.csv")
analyst_classification_report

,Precision,Recall,F1-score
Buy,0.4431,0.6847,0.538
Hold,0.2830,0.3355,0.307
Sell,0.3074,0.0087,0.017


---
Making a Latex table to illustrate company presence in the investment pool for the Appendix

In [1]:
import pandas as pd
import numpy as np
analysis_window = pd.read_csv("../results/analysis_window_by_cik.csv", dtype={"cik": str})
analysis_window

,cik,first_date,last_date,security
0,0000001750,02-2000,05-2024,AAR CORP.
1,0000001800,12-1999,12-2023,Abbott Laboratories
2,0000002488,04-2000,12-2023,Advanced Micro Devices
3,0000002969,03-2000,09-2024,Air Products
4,0000003453,03-2000,12-2023,"Matson, Inc."
...,...,...,...,...
1478,0001993004,12-1999,12-2023,NorthWestern Corporation
1479,0001996862,09-2001,12-2023,Bunge Global
1480,0001999001,12-1999,12-2023,Six Flags
1481,0002012383,12-1999,12-2023,BlackRock


In [2]:
analysis_window.rename(columns={"cik": "CIK", "security": "Security", "first_date": "First Date", "last_date": "Last Date"}, inplace=True)
analysis_window.to_csv("../results/analysis_window_by_cik_formatted.csv", index=False)

In [22]:
df = pd.read_csv("../results/analysis_window_by_cik_formatted.csv", dtype={"CIK": str})
df.drop(columns=["Security"], inplace=True)
# Convert date columns to datetime for sorting
df["First Date"] = pd.to_datetime(df["First Date"]).dt.to_period('M')
df["Last Date"] = pd.to_datetime(df["Last Date"]).dt.to_period('M')
df = df.sort_values(by=["First Date", "Last Date"]).reset_index(drop=True)
n_chunks = 5

chunk_size = int(np.ceil(len(df) / n_chunks))
chunks = [df.iloc[i:i + chunk_size].reset_index(drop=True) for i in range(0, len(df), chunk_size)]
combined_df = pd.concat([chunks[0], chunks[1], chunks[2], chunks[3], chunks[4]], axis=1)
combined_df = combined_df.dropna().reset_index(drop=True)
combined_df.to_csv('../results/analysis_window_in_chunks.csv', index=False)

C:\Users\benny\AppData\Local\Temp\ipykernel_18024\2072643922.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["First Date"] = pd.to_datetime(df["First Date"]).dt.to_period('M')
C:\Users\benny\AppData\Local\Temp\ipykernel_18024\2072643922.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Last Date"] = pd.to_datetime(df["Last Date"]).dt.to_period('M')


In [24]:
combined_df

,CIK,First Date,Last Date,CIK,First Date,Last Date,CIK,First Date,Last Date,CIK,First Date,Last Date,CIK,First Date,Last Date
0,0000009092,1999-03,2023-12,0001023128,1999-12,2023-12,0000277135,2000-03,2023-12,0000817720,2002-03,2024-06,0001561894,2013-06,2023-12
1,0000081362,1999-03,2023-12,0001025996,1999-12,2023-12,0000310142,2000-03,2023-12,0001158449,2002-04,2023-12,0001562476,2013-06,2023-12
2,0000089439,1999-03,2023-12,0001031203,1999-12,2023-12,0000310158,2000-03,2023-12,0001326380,2002-05,2024-02,0001569187,2013-06,2023-12
3,0000090498,1999-03,2023-12,0001031296,1999-12,2023-12,0000310764,2000-03,2023-12,0000097134,2002-06,2023-12,0001571949,2013-06,2023-12
4,0000094344,1999-03,2023-12,0001034670,1999-12,2023-12,0000313616,2000-03,2023-12,0000101199,2002-06,2023-12,0001024305,2013-06,2024-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,0001013857,1999-12,2023-12,0000103730,2000-03,2023-12,0001144215,2002-02,2024-08,0001555280,2013-03,2023-12,0001959348,2023-12,2023-12
291,0001013934,1999-12,2023-12,0000104918,2000-03,2023-12,0000903129,2002-03,2023-12,0001561680,2013-03,2023-12,0001968915,2023-12,2023-12
292,0001014739,1999-12,2023-12,0000105016,2000-03,2023-12,0001071739,2002-03,2023-12,0000834365,2013-06,2023-12,0001974138,2023-12,2023-12
293,0001019849,1999-12,2023-12,0000106535,2000-03,2023-12,0001109242,2002-03,2023-12,0001478242,2013-06,2023-12,0001967649,2023-12,2024-09


In [ ]:
import pandas as pd
df = pd.read_csv('../results/analysis_window_in_chunks.csv', dtype=str)
df = df.sort_values(by = df.columns[1])  
df = df.fillna('---')

# 5 chunks with 3 columns each (CIK, Start, End) plus some extra space between them
col_spec = "ccc @{\hspace{6pt}} ccc @{\hspace{6pt}} ccc @{\hspace{6pt}} ccc @{\hspace{6pt}} ccc"

# Latex longtable header with 5 chunks (15 columns total)
latex_header = r"""
{\tiny
\setlength{\tabcolsep}{1.2pt}
\begin{longtable}{""" + col_spec + r"""}
\caption{Analysis Window in Chunks (5-way Horizontal Split)} \label{tab:analysis_chunks} \\
\toprule
\textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} \\
\midrule
\endfirsthead
\multicolumn{15}{c}{{\bfseries \tablename\ \thetable{} -- continued}} \\
\toprule
\textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} & \textbf{CIK} & \textbf{Start} & \textbf{End} \\
\midrule
\endhead
\bottomrule
\endfoot
"""

# Constructing the rows for the longtable
rows = []
for _, row in df.iterrows():
    # Only take the first 15 columns
    row_data = row.values[:15] 
    rows.append(" & ".join(row_data) + r" \\")

# Combining into final LaTeX code
full_latex = latex_header + "\n".join(rows) + "\n\\end{longtable}\n}"

# Save to file
with open("../results/formatted_table.tex", "w") as f:
    f.write(full_latex)